<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C8-Common_Trojaning_Tasks_On_Windows/Pythonic_Shellcode_Execution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>